In [1]:
!git clone https://github.com/zhouhaoyi/Informer2020.git
!git clone https://github.com/zhouhaoyi/ETDataset.git
!ls

fatal: destination path 'Informer2020' already exists and is not an empty directory.
fatal: destination path 'ETDataset' already exists and is not an empty directory.
ETDataset  Informer2020  informer_checkpoints  results


In [ ]:
import os

os.environ['mistral'] = 'API Key'

In [3]:
import sys
if not 'Informer2020' in sys.path:
    sys.path += ['Informer2020']

In [4]:
%%capture
!pip install mistralai
!pip install -r ./Informer2020/requirements.txt

In [30]:
from utils.tools import dotdict
from exp.exp_informer import Exp_Informer
import torch
from mistralai import Mistral
import copy
import numpy as np
import pandas as pd
from tqdm import tqdm
import random

In [68]:
main_args = {
    'model': 'informer',
    'data': 'ETTh1',
    'root_path': './ETDataset/ETT-small/',
    'data_path': 'ETTh1.csv',
    'features': 'M',
    'target': 'OT',
    'freq': 'h',
    'checkpoints': './informer_checkpoints',
    'seq_len': 96,
    'label_len': 48,
    'pred_len': 24,
    'enc_in': 7,
    'dec_in': 7,
    'c_out': 7,

    'des': 'exp',
    'use_gpu': True,
    'gpu': 0,
    'use_multi_gpu': False,
    'devices': '0',
    'detail_freq': 'h',

    'loss': 'mse',
    'lradj': 'type1',
    'use_amp': False,
    'num_workers': 0,

    'distil': True,
    'output_attention': False,
    'mix': True,

    'dropout': 0.05,
    'padding': 0,
    'activation': 'gelu',
    'patience': 3,
    "batch_size": 32,
    "itr": 2,
    "learning_rate": 1e-4,
    "attn": "full",
    "n_heads": 8
    
}
# dotdict(main_args).c_out

In [72]:
search_space = {
    'factor': [i for i in range(4, 10, 2)],
    'd_model': [256, 512, 1024],
    # 'n_heads': [4],
    'e_layers': [2, 4],
    'd_layers': [2, 4],
    'd_ff': [1024, 2048],
    
    # 'attn': ['prob', "full"],
    'embed': ['timeF', "fixed", "learned"],
    'learning_rate': [1e-4],
    'train_epochs': [6, 8, 10],
}

search_space

{'factor': [4, 6, 8],
 'd_model': [256, 512, 1024],
 'e_layers': [2, 4],
 'd_layers': [2, 4],
 'd_ff': [1024, 2048],
 'embed': ['timeF', 'fixed', 'learned'],
 'learning_rate': [0.0001],
 'train_epochs': [6, 8, 10]}

In [73]:
from itertools import product

def create_architecture_space(space, main_args):
    # Получаем список ключей в фиксированном порядке
    keys = list(space.keys())
    
    # Получаем списки индексов для каждого параметра
    index_ranges = [range(len(space[key])) for key in keys]
    
    # Создаём словарь с закодированными ключами
    config_dict = {}
    
    for indices in product(*index_ranges):
        # Формируем строку индексов (например, '000000000000')
        index_str = ''.join(str(i) for i in indices)
        
        # Формируем словарь значений для текущей комбинации
        config = {key: space[key][idx] for key, idx in zip(keys, indices)}
        
        config_dict[index_str] = config
        config_dict[index_str].update(main_args)
        
    return config_dict


architecture  = create_architecture_space(
    space = search_space,
    main_args = main_args
)

In [74]:
len(architecture)

648

## Шаг 1. Накапливание статистики

In [ ]:
def get_history(architecture_dict, n = 50):
    sample = random.sample(list(architecture_dict.keys()), k = n)

    history = {"architecture_code": [], "mse": []}

    for arch in tqdm(sample):
        config = dotdict(architecture_dict[arch])
        Exp = Exp_Informer
        exp = Exp(config)

        # train
        res_train = exp.train(arch)
        
        # test
        res_test = exp.test(arch)
        
        torch.cuda.empty_cache()
    
        metrics = np.load(f'/kaggle/working/results/{arch}/metrics.npy')
        mae, mse, rmse, mape, mspe = metrics
        
        history["architecture_code"].append(arch)
        history["mse"].append(mse)

    return pd.DataFrame(history)


history = get_history(
    architecture_dict = architecture,
    n = 20
)

In [120]:
history.to_csv("/kaggle/working/history.csv", index = False)

## Шаг 2. Обращаемся к LLM с накопленными сэмплами данных о некоторых архитектурах, чтобы она определила, какая архитектура лучше

In [129]:
import json

In [121]:
print(history.to_markdown() )

|    |   architecture_code |      mse |
|---:|--------------------:|---------:|
|  0 |            02111102 | 0.8306   |
|  1 |            11010202 | 0.862981 |
|  2 |            11101102 | 0.724865 |
|  3 |            01100002 | 0.689152 |
|  4 |            11111000 | 0.577649 |
|  5 |            02011201 | 0.819256 |
|  6 |            00011001 | 0.666195 |
|  7 |            12100201 | 0.819361 |
|  8 |            22000102 | 1.00801  |
|  9 |            10100000 | 0.745192 |
| 10 |            22001102 | 0.969087 |
| 11 |            01111002 | 0.77152  |
| 12 |            22110000 | 0.649816 |
| 13 |            20010001 | 0.79892  |
| 14 |            12010102 | 0.852477 |
| 15 |            12110002 | 0.737097 |
| 16 |            20000101 | 0.780946 |
| 17 |            02111002 | 0.511465 |
| 18 |            11000201 | 0.717734 |
| 19 |            11000102 | 0.889464 |


In [122]:
system = """Ты ассистент для поиска лучшей архитектуры нейоросети для задачи Time-series-forecasting.
На вход ты получаешь на вход 
1) N закодированный список архитектур;
2) N метрик MSE на валидации для каждой архитекутры;
3) K закодированных доступных архитекутр.

Код архитектуры это набор индексов для каждого параметра из search_space
search_space = {'factor': [4, 6, 8],
 'd_model': [256, 512, 1024],
 'e_layers': [2, 4],
 'd_layers': [2, 4],
 'd_ff': [1024, 2048],
 'embed': ['timeF', 'fixed', 'learned'],
 'learning_rate': [0.0001],
 'train_epochs': [6, 8, 10]
 }


Твоя задача прогнозировать код лучшей архитектуры из списка доступных архитектур основываясь на истории части кандидатов в формате JSON-object. 
Прогнозируй кандидата из списка доступных кандидатов. Не бери кандидата из истории
Формат JSON-object
{"architecture": str}
"""

In [123]:
user_prompt = """
Пожалуйста, проанализируйте историю, оцените кандидатов и сделай прогноз того кандидата, 
кто покажет самые лучший результат из всего списка кандидатов и верни json в формате {{"architecture": str}}
Прогнозируй кандидата из списка доступных кандидатов. Не бери кандидата из истории
 
History:
{history}

Candidate: 
{candidate}
"""

In [124]:
hist = history.to_markdown()
cand = "\n".join(list(architecture.keys()))

In [125]:
user = user_prompt.format(
    history = hist,
    candidate = cand
)

print(user[:1000])


Пожалуйста, проанализируйте историю, оцените кандидатов и сделай прогноз того кандидата, 
кто покажет самые лучший результат из всего списка кандидатов и верни json в формате {"architecture": str}
Прогнозируй кандидата из списка доступных кандидатов. Не бери кандидата из истории
 
History:
|    |   architecture_code |      mse |
|---:|--------------------:|---------:|
|  0 |            02111102 | 0.8306   |
|  1 |            11010202 | 0.862981 |
|  2 |            11101102 | 0.724865 |
|  3 |            01100002 | 0.689152 |
|  4 |            11111000 | 0.577649 |
|  5 |            02011201 | 0.819256 |
|  6 |            00011001 | 0.666195 |
|  7 |            12100201 | 0.819361 |
|  8 |            22000102 | 1.00801  |
|  9 |            10100000 | 0.745192 |
| 10 |            22001102 | 0.969087 |
| 11 |            01111002 | 0.77152  |
| 12 |            22110000 | 0.649816 |
| 13 |            20010001 | 0.79892  |
| 14 |            12010102 | 0.852477 |
| 15 |            12110002 |

In [132]:
def fit_model(architecture_dict, history, n_iter = 10):
    api_key = os.environ["mistral"]
    model = "mistral-large-latest"
    client = Mistral(api_key=api_key)
    
    history_ = history.copy()
    result = {"architecture_code": [], "mse": []}
    
    for i in tqdm(range(n_iter)):
        user = user_prompt.format(
            history = history_.to_markdown(),
            candidate = "\n".join(list(architecture_dict.keys()))
        )
        
        
        messages = [
             {
                "role": "system",
                "content": system,
            },
            {
                "role": "user",
                "content": user
            }
        ]
        chat_response = client.chat.complete(
              model = model,
              messages = messages,
              response_format = {
                  "type": "json_object",
              }
        )

        new_arch = json.loads(chat_response.choices[0].message.content)["architecture"]

        hist = {"architecture_code": [], "mse": []}
        config = dotdict(architecture_dict[new_arch])
        Exp = Exp_Informer
        exp = Exp(config)

        # train
        res_train = exp.train(new_arch)
        
        # test
        res_test = exp.test(new_arch)
        torch.cuda.empty_cache()
        
        
        metrics = np.load(f'/kaggle/working/results/{new_arch}/metrics.npy')
        mae, mse, rmse, mape, mspe = metrics
        
        hist["architecture_code"].append(new_arch)
        hist["mse"].append(mse)
        
        result["architecture_code"].append(new_arch)
        result["mse"].append(mse)

        history_ = pd.concat([history_, pd.DataFrame(hist)], ignore_index = True)
        
         
    return pd.DataFrame(result), history_



res, hist = fit_model(architecture, history, n_iter = 10)

  0%|          | 0/10 [00:00<?, ?it/s]

Use GPU: cuda:0
train 8521
val 2857
test 2857
	iters: 100, epoch: 1 | loss: 0.3798703
	speed: 0.0982s/iter; left time: 146.9719s
	iters: 200, epoch: 1 | loss: 0.2870650
	speed: 0.0985s/iter; left time: 137.6163s
Epoch: 1 cost time: 26.188100814819336
Epoch: 1, Steps: 266 | Train Loss: 0.3744437 Vali Loss: 0.7535023 Test Loss: 0.7024012
Validation loss decreased (inf --> 0.753502).  Saving model ...
Updating learning rate to 0.0001
	iters: 100, epoch: 2 | loss: 0.2411661
	speed: 0.2204s/iter; left time: 271.3494s
	iters: 200, epoch: 2 | loss: 0.2024616
	speed: 0.0987s/iter; left time: 111.6475s
Epoch: 2 cost time: 26.203690767288208
Epoch: 2, Steps: 266 | Train Loss: 0.2258080 Vali Loss: 0.8176426 Test Loss: 0.8004386
EarlyStopping counter: 1 out of 3
Updating learning rate to 5e-05
	iters: 100, epoch: 3 | loss: 0.1538645
	speed: 0.2174s/iter; left time: 209.7671s
	iters: 200, epoch: 3 | loss: 0.1370364
	speed: 0.0985s/iter; left time: 85.2341s
Epoch: 3 cost time: 26.19013810157776
Epoc

 10%|█         | 1/10 [02:10<19:34, 130.51s/it]

test shape: (89, 32, 24, 7) (89, 32, 24, 7)
test shape: (2848, 24, 7) (2848, 24, 7)
mse:0.7024012207984924, mae:0.641295313835144
Use GPU: cuda:0
train 8521
val 2857
test 2857
	iters: 100, epoch: 1 | loss: 0.3916713
	speed: 0.0980s/iter; left time: 146.6436s
	iters: 200, epoch: 1 | loss: 0.2705301
	speed: 0.0986s/iter; left time: 137.7103s
Epoch: 1 cost time: 26.172691345214844
Epoch: 1, Steps: 266 | Train Loss: 0.3739904 Vali Loss: 0.6958745 Test Loss: 0.5930820
Validation loss decreased (inf --> 0.695875).  Saving model ...
Updating learning rate to 0.0001
	iters: 100, epoch: 2 | loss: 0.2246305
	speed: 0.2229s/iter; left time: 274.3924s
	iters: 200, epoch: 2 | loss: 0.2194930
	speed: 0.0989s/iter; left time: 111.8640s
Epoch: 2 cost time: 26.28778052330017
Epoch: 2, Steps: 266 | Train Loss: 0.2223120 Vali Loss: 0.7732815 Test Loss: 0.6458002
EarlyStopping counter: 1 out of 3
Updating learning rate to 5e-05
	iters: 100, epoch: 3 | loss: 0.1298392
	speed: 0.2182s/iter; left time: 210.5

 20%|██        | 2/10 [04:22<17:31, 131.46s/it]

...

test shape: (89, 32, 24, 7) (89, 32, 24, 7)
test shape: (2848, 24, 7) (2848, 24, 7)
mse:0.7395888566970825, mae:0.6655644178390503
Use GPU: cuda:0
train 8521
val 2857
test 2857
	iters: 100, epoch: 1 | loss: 0.4128161
	speed: 0.0981s/iter; left time: 146.8562s
	iters: 200, epoch: 1 | loss: 0.3128939
	speed: 0.0988s/iter; left time: 137.9772s
Epoch: 1 cost time: 26.2197105884552
Epoch: 1, Steps: 266 | Train Loss: 0.3708971 Vali Loss: 0.7082134 Test Loss: 0.7195722
Validation loss decreased (inf --> 0.708213).  Saving model ...
Updating learning rate to 0.0001
	iters: 100, epoch: 2 | loss: 0.2648109
	speed: 0.2210s/iter; left time: 272.0908s
	iters: 200, epoch: 2 | loss: 0.1894698
	speed: 0.0989s/iter; left time: 111.8022s
Epoch: 2 cost time: 26.265047311782837
Epoch: 2, Steps: 266 | Train Loss: 0.2254133 Vali Loss: 0.7771211 Test Loss: 0.7088330
EarlyStopping counter: 1 out of 3
Updating learning rate to 5e-05
	iters: 100, epoch: 3 | loss: 0.1449258
	speed: 0.2176s/iter; left time: 209.9

 90%|█████████ | 9/10 [20:13<02:13, 133.07s/it]

test shape: (89, 32, 24, 7) (89, 32, 24, 7)
test shape: (2848, 24, 7) (2848, 24, 7)
mse:0.7195721864700317, mae:0.6466553211212158
Use GPU: cuda:0
train 8521
val 2857
test 2857
	iters: 100, epoch: 1 | loss: 0.4200968
	speed: 0.0982s/iter; left time: 147.0085s
	iters: 200, epoch: 1 | loss: 0.3428781
	speed: 0.0988s/iter; left time: 137.9624s
Epoch: 1 cost time: 26.23542881011963
Epoch: 1, Steps: 266 | Train Loss: 0.3809498 Vali Loss: 0.7209316 Test Loss: 0.6914836
Validation loss decreased (inf --> 0.720932).  Saving model ...
Updating learning rate to 0.0001
	iters: 100, epoch: 2 | loss: 0.2610522
	speed: 0.2205s/iter; left time: 271.4474s
	iters: 200, epoch: 2 | loss: 0.1898847
	speed: 0.0987s/iter; left time: 111.5809s
Epoch: 2 cost time: 26.207016468048096
Epoch: 2, Steps: 266 | Train Loss: 0.2255620 Vali Loss: 0.7653608 Test Loss: 0.7583305
EarlyStopping counter: 1 out of 3
Updating learning rate to 5e-05
	iters: 100, epoch: 3 | loss: 0.1480596
	speed: 0.2175s/iter; left time: 209.

100%|██████████| 10/10 [22:24<00:00, 134.47s/it]

test shape: (89, 32, 24, 7) (89, 32, 24, 7)
test shape: (2848, 24, 7) (2848, 24, 7)
mse:0.6914834976196289, mae:0.6413654685020447


In [133]:
res

,architecture_code,mse
0,11111000,0.702401
1,11111000,0.593082
2,01111000,0.660586
3,01111000,0.606475
4,01111000,0.587007
5,01111000,0.682350
6,01111000,0.628665
7,01111000,0.739589
8,01111000,0.719572
9,01111000,0.691483


In [126]:
api_key = os.environ["mistral"]
model = "mistral-large-latest"

client = Mistral(api_key=api_key)
messages = [
     {
        "role": "system",
        "content": system,
    },
    {
        "role": "user",
        "content": user
    }
]
chat_response = client.chat.complete(
      model = model,
      messages = messages,
      response_format = {
          "type": "json_object",
      }
)

In [128]:
json.loads(chat_response.choices[0].message.content)

{'architecture': '01111100'}